### codenames
- scan in screenshot from codenames
- extract the game box, words and colors
- optimize hint selection for each color

```
conda install -c conda-forge pytesseract
```

In [30]:
import numpy as np
import cv2 as cv
import os
from PIL import Image
import pytesseract

In [31]:
import matplotlib.pyplot as plt
from collections import Counter

In [32]:
wd = r'C:\Users\CHONGW2\OneDrive - The Toronto-Dominion Bank\Projects\skunkworks\opencv_playing'

In [ ]:
image = cv.imread(os.path.join(wd, 'data', 'codenames.jpg'))
cv.imshow("image", image)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

In [ ]:
edges = cv.Canny(gray,50,150)
cv.imshow("image", edges)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
contours, hierarchy = cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

In [ ]:
contours_img = cv.drawContours(image, contours, -1, (0,255,0), 3)
cv.imshow("image", contours_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
red = image.copy()
red[:,:,0] = 0
red[:,:,1] = 0
cv.imshow("image", red)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
red_gray = cv.cvtColor(red, cv.COLOR_BGR2GRAY)

In [ ]:
ret, thresh_red = cv.threshold(red_gray, int(red_gray.mean()), 255, cv.THRESH_BINARY)
cv.imshow("image", thresh_red)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
blur = cv.blur(red_gray,(5,5))
cv.imshow("image", thresh_red)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
im2, contours, hierarchy = cv.findContours(thresh_red, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

In [ ]:
image_copy = image.copy()
contours_img = cv.drawContours(image_copy, contours, -1, (0,255,0), 3)
cv.imshow("image", contours_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
color_array = image.reshape(-1,3)

In [ ]:
cv.imshow("image", image)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
cv.imshow("image", gray)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
ret, thresh1 = cv.threshold(gray, int(gray.mean()), 255, cv.THRESH_BINARY)
cv.imshow("image", thresh1)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
gray.mean()

In [ ]:
gray.std()/np.sqrt(len(gray))

In [ ]:
gray_threshold = cv.inRange(gray, int(gray.mean() - 2*gray.std()/np.sqrt(len(gray))), int(gray.mean() + 2*gray.std()/np.sqrt(len(gray))))
cv.imshow("image", gray_threshold)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
thresh1.shape

In [ ]:
(thresh1==0).mean(1).shape

In [ ]:
avg_white = (thresh1==0).mean()

In [ ]:
std_white = (thresh1==0).std()

In [ ]:
std_white/np.sqrt(len(thresh1))

In [ ]:
x_avg_white, y_avg_white = (thresh1==0).mean(0), (thresh1==0).mean(1)

In [ ]:
vert_lines = np.where((x_avg_white < (avg_white + 2*std_white/np.sqrt(len(thresh1)))) & (x_avg_white > (avg_white - 2*std_white/np.sqrt(len(thresh1)))))[0]

In [ ]:
vert_lines

In [ ]:
color = (255, 0, 0) 
thickness = 9
for x in vert_lines:
    thresh1 = cv.line(thresh1, (x,0), (x,thresh1.shape[1]), color, thickness)

In [ ]:
plt.imshow(thresh1, cmap='gray')
plt.show()

In [ ]:
cv.imshow("image", thresh1)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
contours, hierarchy = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

In [ ]:
dims = [arr.shape[0] for arr in contours]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# running kmeans for a specific rule
X = np.array(dims).reshape((-1,1))

sse = []
for i in np.arange(2, 30, 1):
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(X)
    sse.append(km.inertia_)

# plot
plt.plot(np.arange(2, 30, 1), sse, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
# Compute the clusters
n_clusters = 100
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
clusters = kmeans.fit_predict(X)

In [ ]:
clusters[X.reshape(-1)==1184]

In [ ]:
X[clusters==71]

In [ ]:
X[clusters==71].shape

In [ ]:
Counter(clusters)

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.scatter(x=X, y=np.repeat(10,len(X)), c=clusters)
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
plt.show()

In [ ]:
contours_img = cv.drawContours(image, contours, -1, (0,255,0), 1)

In [ ]:
cv.imshow("image", contours_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
len(contours)

In [ ]:
contours2 = [arr for arr in contours if arr.shape==(1184, 1, 2)]

In [ ]:
contours3 = [arr for arr in contours if arr.shape[0] in X[clusters==52].reshape(-1)]

In [ ]:
image = cv.imread(os.path.join(wd, 'data', 'codenames.jpg'))

In [ ]:
contours_img = cv.drawContours(image, contours3, -1, (0,255,0), 3)
cv.imshow("image", contours_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
Counter([arr.shape for arr in contours])

In [ ]:
(thresh1==0).mean(0) > avg_white

In [ ]:
blur = cv.bilateralFilter(thresh1,9,75,75)

In [ ]:
cv.imshow("image", thresh1)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
frame_threshold = cv.inRange(gray, int(gray.mean() - gray.std()), int(gray.mean() + gray.std()))

In [ ]:
cv.imshow("image", frame_threshold)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
color_array.mean(0)

### trying tesseract

In [33]:
from PIL import Image
import pytesseract

In [34]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\CHONGW2\AppData\Local\Tesseract-OCR\tesseract.exe'

In [35]:
filename = os.path.join(wd, 'data', 'gray.png')

In [36]:
# cv.imwrite(filename, thresh1)

NameError: name 'thresh1' is not defined

In [38]:
text = pytesseract.image_to_string(Image.open(filename))
# os.remove(filename)
print(text.split())

['MINT', 'HORSE', 'GREEN', 'STICK', 'BELT', 'BUFFALO', 'PYRAMID', 'SHOT', 'CLIFF', 'LUCK', 'CIRCLE', 'GAS', 'GLASS', 'CHANGE', 'ALPS', 'SUIT', 'ROCK', 'KID', 'PRINCESS', 'DRAFT', 'FAIR', 'SMUGGLER', 'CRASH', 'STAR']


In [39]:
pytesseract.image_to_boxes(Image.open(filename))

'M 169 1212 192 1250 0\nI 192 1212 213 1250 0\nN 221 1212 243 1250 0\nT 248 1212 265 1250 0\nH 155 924 176 962 0\nO 183 923 208 963 0\nR 215 924 235 962 0\nS 227 923 248 963 0\nE 241 923 278 963 0\nG 155 635 180 675 0\nR 186 636 207 674 0\nE 213 636 227 674 0\nE 234 636 248 674 0\nN 254 636 277 674 0\nS 166 347 184 387 0\nT 187 348 215 386 0\nI 202 347 223 387 0\nC 222 347 240 387 0\nK 247 348 267 386 0\nB 179 60 197 98 0\nE 204 60 217 98 0\nL 224 60 238 98 0\nT 239 60 257 98 0\nB 582 1212 600 1250 0\nU 607 1211 628 1250 0\nF 636 1212 649 1250 0\nF 656 1212 669 1250 0\nA 672 1212 696 1250 0\nL 701 1212 715 1250 0\nO 719 1211 744 1251 0\nP 575 924 594 962 0\nY 596 924 620 962 0\nR 625 924 645 962 0\nA 650 924 674 962 0\nM 678 924 711 962 0\nI 717 924 723 962 0\nD 730 924 752 962 0\nS 614 635 632 675 0\nH 638 636 659 674 0\nO 666 635 691 675 0\nT 695 636 713 674 0\nC 617 347 635 387 0\nL 633 347 654 387 0\nI 642 348 667 386 0\nF 674 348 688 386 0\nF 694 348 708 386 0\nL 618 60 632 98 0\n

### trying optimization

In [2]:
red = np.array(['horse', 'belt', 'shot', 'change', 'suit', 'kid', 'princess', 'crash'])
blue = np.array(['mint', 'stick', 'buffalo', 'cliff', 'luck', 'glass', 'alps', 'smuggler', 'star'])
neutral = np.array(['green', 'pyramid', 'circle', 'gas', 'rock', 'draft', 'fair'])
black = np.array(['pupil'])

In [3]:
everything = np.concatenate((red, blue, neutral, black))
everything

array(['horse', 'belt', 'shot', 'change', 'suit', 'kid', 'princess',
       'crash', 'mint', 'stick', 'buffalo', 'cliff', 'luck', 'glass',
       'alps', 'smuggler', 'star', 'green', 'pyramid', 'circle', 'gas',
       'rock', 'draft', 'fair', 'pupil'], dtype='<U8')

In [70]:
# get pretrained model
import gensim.downloader as api
# api.load('glove-wiki-gigaword-100', return_path=True)
# glove-twitter-100
model = api.load("glove-twitter-100")
model.most_similar("glass")


[==================================================] 100.0% 387.1/387.1MB downloaded


[('bottle', 0.7187241315841675),
 ('crystal', 0.6808942556381226),
 ('light', 0.6784807443618774),
 ('wine', 0.6726185083389282),
 ('lamp', 0.6709625720977783),
 ('wall', 0.666566014289856),
 ('green', 0.6661511063575745),
 ('silver', 0.6573828458786011),
 ('cherry', 0.6446923017501831),
 ('white', 0.6388335227966309)]

In [14]:
model.most_similar_cosmul(positive=['woman', 'king'])

[('father', 0.7581130266189575),
 ('mother', 0.7420350313186646),
 ('son', 0.7351132035255432),
 ('daughter', 0.7230614423751831),
 ('man', 0.721849262714386),
 ('queen', 0.7136995792388916),
 ('wife', 0.7132636904716492),
 ('her', 0.7127906084060669),
 ('brother', 0.7064565420150757),
 ('his', 0.7014669179916382)]

In [16]:
(model['king']+model['woman'])/2

array([ 0.16149   ,  0.66717005, -0.58863497, -0.2586555 ,  1.07098   ,
        0.60501003, -0.26059002,  0.52718   , -0.13121   , -0.28098   ,
        0.062937  ,  0.388285  ,  0.13595551, -0.21023   ,  0.760785  ,
        0.218185  , -0.713745  ,  0.05911499, -0.286263  ,  0.45073003,
       -0.17094499,  0.821175  , -0.0905965 ,  0.197255  ,  0.55234003,
       -2.44845   , -0.92182505, -0.369505  , -0.073295  , -0.00870001,
        2.14235   , -0.158225  , -0.47244   ,  0.35605145,  0.33041   ,
        0.1294865 ,  0.159915  , -0.110197  , -0.25370997, -0.336685  ,
        0.07463999,  0.79486495, -0.26376998, -0.0967625 ,  0.20302501,
       -0.03451002, -0.57157004, -1.4621999 , -0.07831   , -0.307545  ],
      dtype=float32)

In [42]:
model.most_similar(positive=['belt'], topn=10)

[('belts', 0.8093633055686951),
 ('conveyer', 0.7111872434616089),
 ('conveyor', 0.685033917427063),
 ('waist', 0.6747647523880005),
 ('grip', 0.6731212139129639),
 ('helmet', 0.6455352306365967),
 ('trunk', 0.6446360945701599),
 ('ring', 0.6412265300750732),
 ('rope', 0.6406635046005249),
 ('heel', 0.639432430267334)]

In [43]:
model[model.most_similar(positive=['belt'], topn=10)]

TypeError: not all arguments converted during string formatting

In [28]:
# averaging
for i in range(len(red)):
    for j in range(i):
        print(red[i], red[j])
        print(model.most_similar(positive=[model[red[i]]+model[red[j]]], topn=3))

belt horse
[('horse', 0.8737381100654602), ('belt', 0.8694759607315063), ('riding', 0.7257648706436157)]
shot horse
[('shot', 0.8469933867454529), ('horse', 0.816696286201477), ('pulled', 0.7947872877120972)]
shot belt
[('shot', 0.8549672365188599), ('belt', 0.8130803108215332), ('pulled', 0.8087259531021118)]
change horse
[('this', 0.8236865997314453), ('horse', 0.808336615562439), ('change', 0.8048089742660522)]
change belt
[('change', 0.8323100805282593), ('belt', 0.8212648034095764), ('control', 0.8034288883209229)]
change shot
[('but', 0.9005899429321289), ('out', 0.8944075703620911), ('put', 0.8942121267318726)]
suit horse
[('suit', 0.8404357433319092), ('horse', 0.8298599720001221), ('dog', 0.7573620676994324)]
suit belt
[('suit', 0.8455564975738525), ('belt', 0.8229615688323975), ('suits', 0.746342658996582)]
suit shot
[('shot', 0.824576199054718), ('pulled', 0.8136223554611206), ('suit', 0.7982337474822998)]
suit change
[('suit', 0.8554710149765015), ('change', 0.8428727388381

### topk
- grabbing topK, removing the most similar (may be a plural, for example)
- pairwise comparison between topK

In [68]:
for i in range(len(red)):
    for j in range(i):
        print(red[i], red[j])
        topK_i = model.most_similar(red[i], topn=20)
        topK_j = model.most_similar(red[j], topn=20)
        topK_i_words = [word for (word, sim) in topK_i if sim < 0.9]
        topK_j_words = [word for (word, sim) in topK_j if sim < 0.9]
        print(topK_i_words)
        print(topK_j_words)
        if len(topK_i_words) * len(topK_j_words) > 0:
            cos_matrix = cosine_similarity(model[topK_i_words], model[topK_j_words])
            word_matrix = np.array(np.meshgrid(topK_i_words, topK_j_words)).T.reshape(len(topK_i_words), len(topK_j_words), 2)
    #         print(word_matrix)
            print(word_matrix[cos_matrix > 0.65]) # perhaps add averaging here
        else:
            print('blah')


belt horse
['belts', 'conveyer', 'conveyor', 'waist', 'grip', 'helmet', 'trunk', 'ring', 'rope', 'heel', 'boot', 'patch', 'strap', 'outer', 'gear', 'control', 'blade', 'neck', 'attached', 'tight']
['horses', 'dog', 'riding', 'bull', 'breeders', 'cat', 'derby', 'ride', 'camel', 'bike', 'thoroughbred', 'racing', 'rides', 'pack', 'pony', 'rode', 'stud', 'hunting', 'cart', 'race']
[['boot' 'pack']
 ['gear' 'pack']]
shot horse
['shots', 'pulled', 'grabbed', 'missed', 'caught', 'off', 'shooting', 'struck', 'behind', 'landed', 'shoot', 'hit', 'ball', 'picked', 'out', 'fired', 'blew', 'threw', 'turned', 'man']
['horses', 'dog', 'riding', 'bull', 'breeders', 'cat', 'derby', 'ride', 'camel', 'bike', 'thoroughbred', 'racing', 'rides', 'pack', 'pony', 'rode', 'stud', 'hunting', 'cart', 'race']
[['behind' 'race']
 ['shoot' 'pack']
 ['picked' 'pack']
 ['out' 'pack']
 ['man' 'dog']]
shot belt
['shots', 'pulled', 'grabbed', 'missed', 'caught', 'off', 'shooting', 'struck', 'behind', 'landed', 'shoot', 

In [50]:
np.array(np.meshgrid(red, red)).T.reshape(len(red), len(red) ,2)

array([[['horse', 'horse'],
        ['horse', 'belt'],
        ['horse', 'shot'],
        ['horse', 'change'],
        ['horse', 'suit'],
        ['horse', 'kid'],
        ['horse', 'princess'],
        ['horse', 'crash']],

       [['belt', 'horse'],
        ['belt', 'belt'],
        ['belt', 'shot'],
        ['belt', 'change'],
        ['belt', 'suit'],
        ['belt', 'kid'],
        ['belt', 'princess'],
        ['belt', 'crash']],

       [['shot', 'horse'],
        ['shot', 'belt'],
        ['shot', 'shot'],
        ['shot', 'change'],
        ['shot', 'suit'],
        ['shot', 'kid'],
        ['shot', 'princess'],
        ['shot', 'crash']],

       [['change', 'horse'],
        ['change', 'belt'],
        ['change', 'shot'],
        ['change', 'change'],
        ['change', 'suit'],
        ['change', 'kid'],
        ['change', 'princess'],
        ['change', 'crash']],

       [['suit', 'horse'],
        ['suit', 'belt'],
        ['suit', 'shot'],
        ['suit', 'change'],
 

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
red

array(['horse', 'belt', 'shot', 'change', 'suit', 'kid', 'princess',
       'crash'], dtype='<U8')

In [12]:
cosine_similarity(model[red], model[red])

array([[1.        , 0.5393989 , 0.40194672, 0.32023942, 0.41400403,
        0.49267405, 0.38919038, 0.3479924 ],
       [0.5393989 , 1.        , 0.41181612, 0.38851792, 0.4141474 ,
        0.36508656, 0.1346004 , 0.36560497],
       [0.40194672, 0.41181612, 1.        , 0.28981543, 0.3350219 ,
        0.4554358 , 0.13844678, 0.5345875 ],
       [0.32023942, 0.38851792, 0.28981543, 1.0000001 , 0.46239328,
        0.36283013, 0.10451277, 0.4045446 ],
       [0.41400403, 0.4141474 , 0.3350219 , 0.46239328, 1.        ,
        0.3365702 , 0.34198192, 0.30726376],
       [0.49267405, 0.36508656, 0.4554358 , 0.36283013, 0.3365702 ,
        1.        , 0.23344147, 0.22632346],
       [0.38919038, 0.1346004 , 0.13844676, 0.10451275, 0.34198195,
        0.23344147, 1.        , 0.36479485],
       [0.3479924 , 0.365605  , 0.5345875 , 0.4045446 , 0.30726376,
        0.22632347, 0.36479485, 1.        ]], dtype=float32)

In [9]:
np.inner(model[red], model[red])

array([[26.432423 , 12.973879 , 11.503617 ,  8.21177  , 10.453274 ,
        12.514881 , 11.429198 ,  9.07983  ],
       [12.973879 , 21.88682  , 10.724874 ,  9.065589 ,  9.515366 ,
         8.438898 ,  3.596855 ,  8.680465 ],
       [11.503617 , 10.724874 , 30.98811  ,  8.046601 ,  9.1590395,
        12.526313 ,  4.402157 , 15.102742 ],
       [ 8.21177  ,  9.065589 ,  8.046601 , 24.876383 , 11.326203 ,
         8.941193 ,  2.9774752, 10.239988 ],
       [10.453274 ,  9.515366 ,  9.1590395, 11.326203 , 24.118952 ,
         8.166828 ,  9.59329  ,  7.658259 ],
       [12.514881 ,  8.438898 , 12.526313 ,  8.941193 ,  8.166828 ,
        24.411648 ,  6.5881243,  5.6750236],
       [11.429198 ,  3.596855 ,  4.402157 ,  2.9774754,  9.593291 ,
         6.5881243, 32.626495 , 10.574831 ],
       [ 9.079831 ,  8.680465 , 15.102742 , 10.239988 ,  7.658259 ,
         5.6750236, 10.574831 , 25.756035 ]], dtype=float32)